# (Winston - operations-research) Example 1 - CHapter 4 

Leather Limited manufactures two types of belts: the deluxe model and the regular model.
Each type requires 1 sq yd of leather. A regular belt requires 1 hour of skilled labor, and
a deluxe belt requires 2 hours. Each week, 40 sq yd of leather and 60 hours of skilled la-
bor are available. Each regular belt contributes \$3 to profit and each deluxe belt, 
\$4. If
we define

$x1$ = number of deluxe belts produced weekly

$x2$ = number of regular belts produced weekly

the appropriate LP is

max z = 4x1 + 3x2

s.t.

x1 = x2 + 40 (Leather constraint)

2x1 = x2 + 60 (Labor constraint)

x1, x2 >= 0

In [17]:
from ortools.init import pywrapinit

pywrapinit.CppBridge.InitLogging('Operations-Research-Winston.ipynb')
cpp_flags = pywrapinit.CppFlags()
cpp_flags.logtostderr = True
cpp_flags.log_prefix = False
pywrapinit.CppBridge.SetFlags(cpp_flags)

: 

: 

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pyomo.core as pyc

model = pyc.ConcreteModel()

model.x1 = pyc.Var(within=pyc.NonNegativeReals)
x1 = model.x1
model.x2 = pyc.Var(within=pyc.NonNegativeReals)
x2 = model.x2

model.Obj = pyc.Objective(expr = 4*x1 +3*x2, sense = pyc.maximize) 

model.Const1 = pyc.Constraint(expr=x1+x2 <= 40)
model.Const2 = pyc.Constraint(expr=2*x1+x2 <= 60)

opm = SolverFactory("glpk")
results = opm.solve(model)

print(f"Objective function = {model.Obj()}")
print(f"x1 = {model.x1()}")
print(f"x2 = {model.x2()}")
print("-----------------")
# print(results)


Objective function = 140.0
x1 = 20.0
x2 = 20.0
-----------------


In [2]:
from ortools.linear_solver import pywraplp

# Create the linear solver with the GLOP backend.
solver: pywraplp.Solver = pywraplp.Solver.CreateSolver('GLOP') # third party solvers require building ortools from source
if not solver:
    raise

x1: pywraplp.Variable = solver.IntVar(0, solver.infinity(), 'x1')
x2: pywraplp.Variable = solver.IntVar(0, solver.infinity(), 'x2')

print('Number of variables =', solver.NumVariables())

# # Create a linear constraint, 0 <= x + y <= 2.
solver.Add(x1+x2 <= 40)
solver.Add(2*x1+x2 <= 60)

print('Number of constraints =', solver.NumConstraints())

solver.Maximize(4*x1 +3*x2) 

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {solver.Objective().Value():.2f}')
    print(f'x1 = {x1.solution_value():.2f}')
    print(f'x2 = {x2.solution_value():.2f}')
    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 2
Number of constraints = 2
Solution:
Objective value = 140.00
x1 = 20.00
x2 = 20.00

Advanced usage:
Problem solved in 0.000000 milliseconds
Problem solved in 2 iterations


# (Winston - operations-research) Example 2 - Page 140

The Dakota Furniture Company manufactures desks, tables, and chairs. The manufacture
of each type of furniture requires lumber and two types of skilled labor: finishing and carpentry. The amount of each resource needed to make each type of furniture is given in
Table 4.
Currently, 48 board feet of lumber, 20 finishing hours, and 8 carpentry hours are avail-
able. A desk sells for \$60, a table for \$30, and a chair for $20. Dakota believes that demand for desks and chairs is unlimited, but at most five tables can be sold. Because the
available resources have already been purchased, Dakota wants to maximize total
revenue.

Resource|Desk|Table|Chair
-|-|-|-
Lumber (board ft)|8|6.5|1.5
Finishing hours|4|2.5|1.5
Carpentry hours|2|1.5|0.5

Defining the decision variables as

- $x1$ = number of desks produced
- $x2$ = number of tables produced
- $x3$ = number of chairs produced

it is easy to see that Dakota should solve the following LP:

max $z = 60x1 + 30x2 + 20x3$

s.t.
- 8x1 + 6x2 + x3 <= 48 (Lumber constraint)
- 4x1 + 2x2 + 1.5x3 <= 20 (Finishing constraint)
- 2x1 + 1.5x2 + 0.5x3 <= 8 (Carpentry constraint)
- x2 <= 5 (Limitation on table demand)
- x1, x2, x3 >= 0

In [3]:
# pending   

# API playground and dummy examples

## Minimize number of time allocation entries for project members

Redistribute total hours to bill for $n$ total users from all teams in a project so that individual
user time entries (forcefully manual) are minimized. 

Inputs:
  - User monthly billable hours to report (excluding holidays, etc.) for every
    client. 
    
    >For proper
    >optimization, knowing each non-billable hours count and day of the month for every
    >user would be necessary. In this example all non-billable hours are effectively aggregated at the end of the month.
  - A month can be composed of either 4 or 5 weeks. Selectable on demand.
  - 

Constraints: 
  - 8 hours/day for reporting application
  - 40 hour/week maximum span of reported project hours. $span_l <= 40.0$


The objective is to minimize total created spans

Example:

4 week month, 3 users with 160,160,150 billable hours to distribute between
projects:

- 1: 140h
- 2: 20h
- 3: 310h

should output something like ( in {hours}#{project} format): 
||Week 1|Week 2|Week 3|Week 4|Spans
--|-|-|-|-|-
user 1| 40#3 | 40#3 | 40#3 | 40#3 | 4
user 3| 40#3 | 40#3 | 40#3 | 30#3 | 4
user 2| 40#1 | 40#1 | 40#1 | 20#1 + 20#2 | 5


In [27]:
import pprint
from ortools.sat.python import cp_model

# Define the input parameters
n_weeks = 4
billing_info = {'1': 140, '2': 20, '3': 310}
hours_per_day = 8
max_weekly_hours = 40

# Initialize the CP-SAT model
model = cp_model.CpModel()

# Create the decision variables
user_hours = [160,160,150] # users [1,2,3]
vars = []
for u in range(len(user_hours)):
    vars.append([])
    for w in range(n_weeks):
        vars[u].append([])
        for p in billing_info.keys():
            vars[u][w].append(model.NewIntVar(0, max_weekly_hours, f'u{u+1}_w{w+1}_p{p}'))

# Add the constraints
for u in range(len(user_hours)):
    # for w in range(n_weeks):
    #     model.Add(sum(vars[u][w]) == sum(billing_info.values()))

    for p in billing_info.keys():
        model.Add(sum(vars[u][w][int(p)-1] for u in range(len(user_hours))) == billing_info[p])

        # model.Add(sum(vars[u][w][int(p)-1][d] for d in range(5)) == hours_per_day * 5)

# Define the objective
spans = []
for u in range(len(user_hours)):
    for w in range(n_weeks):
        user_span = model.NewIntVar(0, max_weekly_hours, f'span_u{u+1}_w{w+1}')
        model.AddMaxEquality(user_span, [vars[u][w][int(p)-1] for p in billing_info.keys()])
        spans.append(user_span)
model.Minimize(sum(spans))

pprint.pprint(spans)
pprint.pprint(vars)

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.INFEASIBLE:
    raise Exception("INFEASIBLE")
if status == cp_model.MODEL_INVALID:
    raise Exception("MODEL_INVALID")

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for u in range(len(user_hours)):
        print(f"user {u+1}:")
        for w in range(n_weeks):
            for p in billing_info.keys():
                project_hours = [solver.Value(vars[u][w][int(p)-1][d]) for d in range(5)]
                print(f"Week {w+1}: {sum(project_hours)}h on project {p} ({', '.join([f'{h}h' for h in project_hours])})")
        print(f"Total span for user {u+1}: {solver.Value(spans[u*n_weeks]):d}")
        print("----")
    print(f"Total span for all users: {solver.ObjectiveValue():d}")
else:
    print("No optimal or feasible solution found.")


[span_u1_w1(0..40),
 span_u1_w2(0..40),
 span_u1_w3(0..40),
 span_u1_w4(0..40),
 span_u2_w1(0..40),
 span_u2_w2(0..40),
 span_u2_w3(0..40),
 span_u2_w4(0..40),
 span_u3_w1(0..40),
 span_u3_w2(0..40),
 span_u3_w3(0..40),
 span_u3_w4(0..40)]
[[[u1_w1_p1(0..40), u1_w1_p2(0..40), u1_w1_p3(0..40)],
  [u1_w2_p1(0..40), u1_w2_p2(0..40), u1_w2_p3(0..40)],
  [u1_w3_p1(0..40), u1_w3_p2(0..40), u1_w3_p3(0..40)],
  [u1_w4_p1(0..40), u1_w4_p2(0..40), u1_w4_p3(0..40)]],
 [[u2_w1_p1(0..40), u2_w1_p2(0..40), u2_w1_p3(0..40)],
  [u2_w2_p1(0..40), u2_w2_p2(0..40), u2_w2_p3(0..40)],
  [u2_w3_p1(0..40), u2_w3_p2(0..40), u2_w3_p3(0..40)],
  [u2_w4_p1(0..40), u2_w4_p2(0..40), u2_w4_p3(0..40)]],
 [[u3_w1_p1(0..40), u3_w1_p2(0..40), u3_w1_p3(0..40)],
  [u3_w2_p1(0..40), u3_w2_p2(0..40), u3_w2_p3(0..40)],
  [u3_w3_p1(0..40), u3_w3_p2(0..40), u3_w3_p3(0..40)],
  [u3_w4_p1(0..40), u3_w4_p2(0..40), u3_w4_p3(0..40)]]]


Exception: INFEASIBLE